In [1]:
!pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 680.9/680.9 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.77.0
    Uninstalling openai-1.77.0:
      Successfully uninstalled openai-1.77.0


In [2]:
from getpass import getpass
import os

# Securely enter your API key
os.environ["OPENAI_API_KEY"] = getpass("🔐 Enter your OpenAI API key: ")

🔐 Enter your OpenAI API key: ··········


In [3]:
from openai import OpenAI

# Create a client using the secure API key
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [4]:
def analyze_linkedin_post(post_text: str) -> dict:
    """
    Sends a LinkedIn post to ChatGPT and returns a dict with:
      - engagement: str
      - user_value: str
      - suggestions: str
    """
    system_prompt = (
        "You are an expert content analyst. "
        "Given a LinkedIn post, provide:\n"
        "1. Engagement: is the post likely engaging? Why/why not?\n"
        "2. User-value: what actionable or informational value does it offer?\n"
        "3. Suggestions: how to improve engagement or clarity.\n"
        "Return your answer in JSON with keys 'engagement', 'user_value', 'suggestions'."
    )

    user_prompt = f"LinkedIn post:\n\"\"\"\n{post_text}\n\"\"\""

    response = client.chat.completions.create(
        model="gpt-4o",  # or use gpt-3.5-turbo
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.3,
    )

    import json
    reply = response.choices[0].message.content
    try:
        return json.loads(reply)
    except json.JSONDecodeError:
        return {"raw": reply}

In [5]:
def main():
    print("🔎 LinkedIn Post Analyzer")
    post = input("Enter your LinkedIn post text:\n> ")
    result = analyze_linkedin_post(post)

    print("\n=== Analysis Result ===")
    if "raw" in result:
        print(result["raw"])
    else:
        print(f"• Engagement: {result['engagement']}")
        print(f"• User Value: {result['user_value']}")
        print(f"• Suggestions: {result['suggestions']}")

if __name__ == "__main__":
    main()

🔎 LinkedIn Post Analyzer
Enter your LinkedIn post text:
> Paper piles, lost files, and endless delays… it’s time to be honest 👇 What’s the WORST thing about managing paper files?  We want to know what frustrates you the most when dealing with paper-based processes. Vote below and see how others feel too!  💡 Explore our Paperless Solutions and say goodbye to the paper chaos: https://lnkd.in/dpAiYUSF  hashtag#InformationManagement hashtag#DigitalTransformation hashtag#PaperlessOffice hashtag#CrownInformat

=== Analysis Result ===
```json
{
  "engagement": "The post is likely engaging due to its interactive nature, inviting users to share their frustrations and vote. The use of emojis and hashtags can also attract attention and increase visibility.",
  "user_value": "The post offers informational value by highlighting a common pain point in managing paper files and suggesting a solution through their paperless solutions link. It also provides an opportunity for users to engage in a discus

In [6]:
system_prompt = (
    "You are a professional content analyst with expertise in business information security, data governance, and records management. "
    "Your task is to analyze a LinkedIn post relevant to this field and provide:\n\n"
    "1. Engagement: Is the post likely to attract attention within professional circles? Does it prompt reactions, comments, or shares?\n"
    "2. User-value: What value does it provide to professionals in information security, compliance, digital archiving, or document lifecycle management?\n"
    "3. Suggestions: What could improve its impact — such as clarity, authority, call to action, or format?\n\n"
    "Respond in a JSON format with three keys: 'engagement', 'user_value', and 'suggestions'."
)

In [7]:
def validate_post_structure(post_text: str) -> dict:
    """
    Validates the structure and length of a LinkedIn post.
    Returns feedback on:
    - word count
    - character count
    - paragraph use
    - presence of hashtags
    - presence of a likely call-to-action
    """
    feedback = {}

    word_count = len(post_text.split())
    char_count = len(post_text)
    paragraphs = post_text.count('\n') + 1
    hashtags = [word for word in post_text.split() if word.startswith('#') or 'hashtag#' in word]
    has_cta = any(phrase in post_text.lower() for phrase in [
        "read more", "contact us", "get in touch", "visit our site", "learn more", "register", "join", "download"
    ])

    # Assess length
    if word_count < 30:
        feedback['length'] = "Post may be too short to be informative or engaging."
    elif word_count > 300:
        feedback['length'] = "Post may be too long; consider shortening for better engagement."
    else:
        feedback['length'] = "Length is within a typical and effective range."

    # Paragraphs
    if paragraphs < 2:
        feedback['paragraphs'] = "Consider breaking content into 2 or more paragraphs for readability."
    else:
        feedback['paragraphs'] = "Paragraph structure looks appropriate."

    # Hashtags
    if not hashtags:
        feedback['hashtags'] = "Consider adding relevant hashtags to improve discoverability."
    else:
        feedback['hashtags'] = f"{len(hashtags)} hashtag(s) detected."

    # Call-to-action
    if not has_cta:
        feedback['call_to_action'] = "No call-to-action detected. Consider adding one to drive engagement."
    else:
        feedback['call_to_action'] = "Call-to-action detected."

    # Summary stats
    feedback['word_count'] = word_count
    feedback['char_count'] = char_count

    return feedback


In [12]:
def main():
    print("🔎 LinkedIn Post Analyzer")
    post = input("Enter your LinkedIn post text:\n> ")

    # Structural validation
    print("\n=== ✅ Pre-check: Structure & Length ===")
    structure_feedback = validate_post_structure(post)
    for key, value in structure_feedback.items():
        print(f"{key.capitalize()}: {value}")

    # GPT-based semantic analysis
    print("\n=== 🤖 GPT Semantic Analysis ===")
    result = analyze_linkedin_post(post)
    if "raw" in result:
        print(result["raw"])
    else:
        print(f"• Engagement: {result['engagement']}")
        print(f"• User Value: {result['user_value']}")
        print(f"• Suggestions: {result['suggestions']}")


In [13]:
if __name__ == "__main__":
    main()

🔎 LinkedIn Post Analyzer
Enter your LinkedIn post text:
> We’re excited to be part of two major events this May that are shaping the future of information management and digital transformation. Come meet our team, explore innovative solutions, and connect with industry professionals.   📍 IRMS Conference   Date: 18-20 May 2025   Location: The Hilton Birmingham Metropole (NEC)   Stand #28   Join us at IRMS 2025 as we explore the evolving landscape of records management. Connect with industry experts and see how Restore Information Management is enabling digital transformation through secure, scalable information solutions.   📍 Procurex England 2025   Date: 21 May 2025   Location: Exhibition Centre Liverpool   Stand #108   Join Restore Information Management at Procurex England 2025 to explore how we're supporting organisations on their journey toward information management journey. Discover practical steps toward digital transformation, improved data efficiency, and smarter information h